In [ ]:
# !pip install efficientnet-pytorch sklearn

In [1]:
import os
import random
import time

import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data.sampler import Sampler
from PIL import Image
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

import models
from dataset_generator import DatasetGenerator

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_size = 224

class_to_idx = {
    'normal': 0,
    'pneumonia': 1,
    'COVID-19': 2
}

In [2]:
print(torch.__version__)
print(torchvision.__version__)

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

1.4.0
0.5.0


In [3]:
image_dir = 'data'
train_csv_file = 'newtrain_split.txt'
test_csv_file = 'test_split.txt'
valid_csv_file = 'valid_split.txt'

In [4]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([
    transforms.RandomOrder([
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15, resample=Image.BILINEAR),
        transforms.RandomResizedCrop(image_size, scale=(0.9, 1.0)),
    ]),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

valid_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transforms = valid_transforms 

In [5]:
# TODO: Load the datasets with DatasetGenerator
train_dir = os.path.join(image_dir, 'train')
train_dataset = DatasetGenerator(train_csv_file, train_dir, transform=train_transforms)
image, label = next(iter(train_dataset))

In [6]:
# TODO: Load the datasets with DatasetGenerator
test_dir = os.path.join(image_dir, 'test')
test_dataset = DatasetGenerator(test_csv_file, test_dir, transform=test_transforms)
image, label = next(iter(test_dataset))

In [7]:
#TODO: Load the datasets with DatasetGenerator
valid_dir = os.path.join(image_dir,'train')
valid_dataset = DatasetGenerator(valid_csv_file, valid_dir, transform=valid_transforms)
image,label = next(iter(valid_dataset))

In [8]:
class WeightedCovidSampler(Sampler):
    def __init__(self, train_noncovid_list, train_covid_list, batch_size, covid_percent):
        self.batch_size = batch_size
        self.train_noncovid_list = np.array(train_noncovid_list)
        self.train_covid_list = np.array(train_covid_list)
        self.covid_percent = covid_percent
        self.covid_size = max(int(batch_size*covid_percent),1) 
        self.size = len(self.train_noncovid_list) + len(self.train_covid_list)

    def __do_batch__(self):
        covid_inds = np.random.choice(self.train_covid_list, size=self.covid_size, replace=False)
        noncovid_inds = np.random.choice(self.train_noncovid_list, size=(self.batch_size-self.covid_size), replace=False)
        batch_inds = np.concatenate((covid_inds,noncovid_inds),axis=None)
        np.random.shuffle(batch_inds)
        
        return batch_inds
        
    def __iter__(self):
        for batch_idx in range(0, self.size, self.batch_size):
            yield self.__do_batch__()

    def __len__(self):
        return self.size

In [9]:
#list of indices for each class 
train_noncovid_list = []
train_covid_list = []

for index in range(len(train_dataset.csv_df)):
    label = train_dataset.csv_df.iloc[index, 2]
    if label == 'COVID-19':
        train_covid_list.append(index)
    else:
        train_noncovid_list.append(index)       

In [10]:
covid_percent = 0.3

```python
covid_size = max(int(batch_size*covid_percent),1)
covid_inds = np.random.choice(np.array(train_covid_list), size=covid_size, replace=False)
len(covid_inds)

noncovid_size = batch_size - covid_size
noncovid_inds = np.random.choice(np.array(train_noncovid_list), size=noncovid_size, replace=False)
len(noncovid_inds)

batch_inds = np.concatenate((covid_inds,noncovid_inds),axis=None)
np.random.shuffle(batch_inds)
print(batch_inds)
```

In [12]:
# TODO: Build and train your network
def load_pretrained_model(arch):
    model_func = getattr(models, arch)
    model = model_func()
    model.arch = arch
    
    return model

In [13]:
# convert labels to tensor
def labels_to_tensor(labels):
    label_indices = [class_to_idx[label] for label in labels]
    label_indices = np.array(label_indices, int)
    label_indices = torch.LongTensor(label_indices)
    return label_indices

labels_to_tensor(['normal', 'pneumonia', 'COVID-19'])

tensor([0, 1, 2])

In [14]:
def validate(model, valid_dataloader, loss_func, device):
    #track accuracy and loss 
    accuracy = 0
    test_loss = 0
    
    with torch.no_grad(): #deactivates requires_grad flag, disables tracking of gradients 
        for images, labels in valid_dataloader: #iterate over images and labels in valid dataset
            labels = labels_to_tensor(labels)
            images, labels = images.to(device), labels.to(device) #move a tensor to a device
            log_ps = model.forward(images) #log form of probabilities for each label
            test_loss += loss_func(log_ps, labels).item() #.item() returns loss value as float, compare prob to actual 
            
            ps = torch.exp(log_ps) #gets rid of log 
            equality = (labels.data == ps.max(dim=1)[1]) #takes highest probability
            accuracy += torch.mean(equality.type(torch.FloatTensor))

    return test_loss, accuracy

In [15]:
# TODO: Do validation on the test set
def test_model(model, test_loader, device):
    #track accuracy, move to device, switch on eval mode
    accuracy = 0
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        for images, labels in iter(test_loader):
            labels = labels_to_tensor(labels)
            images, labels = images.to(device), labels.to(device) #move a tensor to a device
            log_ps = model.forward(images)
            ps = torch.exp(log_ps)
            
            equality = (labels.data == ps.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
        model_accuracy = accuracy/len(test_loader)
        model.accuracy = model_accuracy
    
    return model.accuracy           

In [16]:
# TODO: Save the checkpoint
def save_checkpoint(checkpoint_path, model):
    checkpoint = {
        #TODO: Save the model arch, accuracy, classifier, class_to_idx
        'arch':model.arch,
        'accuracy':model.accuracy,
        'class_to_idx':class_to_idx,
        'state_dict':model.state_dict()
    }
    torch.save(checkpoint, checkpoint_path)
    print('Saved the trained model: %s' % checkpoint_path)

In [17]:
# TODO: Using the image datasets and the trainforms, define the dataloaders
batch_size = 64
weighted_covid_sampler = WeightedCovidSampler(train_noncovid_list,train_covid_list,batch_size,covid_percent)
train_loader = DataLoader(train_dataset, batch_sampler=weighted_covid_sampler, num_workers=16)
dataloaders = {"train": train_loader,
               "test": DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=True),
              "valid":DataLoader(valid_dataset, batch_size=batch_size, num_workers=16, shuffle=True)}

```python
for _, labels in train_loader:
    label_cnt_dict = {
        'normal': 0,
        'pneumonia': 0,
        'COVID-19': 0
    }

    for label in labels:
        label_cnt_dict[label] += 1
    print(label_cnt_dict)
```

In [19]:
device = 'cpu'
cuda = torch.cuda.is_available()
if cuda:
    device = 'cuda'
device

'cuda'

In [20]:
available_models = [m for m in dir(models) if 'Net' in m and 'Model' not in m]
available_models

['DenseNet121',
 'DenseNet169',
 'DenseNet201',
 'EfficientNet4',
 'EfficientNet5',
 'EfficientNet6',
 'ResNet101',
 'ResNet34',
 'ResNet50']

In [21]:
arch = 'EfficientNet5'
pretrained_model = load_pretrained_model(arch)
optimizer = optim.Adam(pretrained_model.get_optimizer_parameters(),
                       lr=0.00001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
criterion = nn.NLLLoss()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /home/ubuntu/.cache/torch/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5


In [22]:
def train(model, loss_func, optimizer, dataloaders, device, epochs, checkpoint_prefix, print_every=20):
    device = torch.device(device)
    model.to(device)

    train_loader = dataloaders['train']
    valid_loader = dataloaders['valid']

    epoch_start = time.time()
    max_acc = 0.0

    # loop to train for number of epochs
    for e in range(epochs):
        running_loss = 0
        batch_start = time.time()
        steps = 0

        for images, labels in train_loader:
            # within each loop, iterate train_loader, and print loss
            label_indices = labels_to_tensor(labels)
            images, labels = images.to(device), label_indices.to(device)
            optimizer.zero_grad()
            log_ps = model.forward(images)
            loss = loss_func(log_ps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            steps += 1

            if steps % print_every == 0:
                model.eval()
                valid_loss, valid_accuracy = validate(model, valid_loader, loss_func, device)
                model.train()
                batch_time = time.time() - batch_start
                print(
                    "Epoch: {}/{}..".format(e + 1, epochs),
                    "Step: {}..".format(steps),
                    "Training Loss: {:.3f}..".format(running_loss / len(train_loader)),
                    "Test Loss: {:.3f}..".format(valid_loss / len(valid_loader)),
                    "Test Accuracy: {:.3f}..".format(valid_accuracy / len(valid_loader)),
                    "Batch Time: {:.3f}, avg: {:.3f}".format(batch_time, batch_time / steps)
                )

        model.eval()
        valid_loss, valid_accuracy = validate(model, valid_loader, loss_func, device)
        model.train()
        epoch_time = time.time() - epoch_start

        if valid_accuracy > max_acc:
            max_acc = valid_accuracy
            model.accuracy = valid_accuracy
            save_checkpoint('%s.pth.tar' % checkpoint_prefix, model)
            print ('Epoch [{}] [save] Accuracy={:.3f} time: {:.3f}, avg: {:.3f}'
                   .format(e + 1, valid_accuracy / len(valid_loader), epoch_time, epoch_time / (e + 1)))
        else:
            print ('Epoch [{}] [----] Accuracy={:.3f} time: {:.3f}, avg: {:.3f}'
                   .format(e + 1, valid_accuracy / len(valid_loader), epoch_time, epoch_time / (e + 1)))

    return model

In [ ]:
#device = 'cpu'
epochs = 2
training_start = time.time()
checkpoint_prefix = os.path.join('checkpoints','%s-%d'%(arch, training_start))
    
trained_model = train(pretrained_model, criterion, optimizer, dataloaders, device, epochs, checkpoint_prefix, print_every=20)
print('%.2f seconds taken for model training' % (time.time() - training_start))

In [ ]:
test_loader = dataloaders['test']
test_accuracy = test_model(trained_model, test_loader, device)
print(test_accuracy)